In [113]:
import tensorflow as tf
import os
import random
import numpy as np
from tqdm import tqdm
import albumentations as A
import cv2

from skimage.io import imread
from skimage.io import imshow
from skimage.util import crop
from skimage.transform import resize
import matplotlib.pyplot as plt
import skimage.filters
from skimage.util import random_noise 
from skimage.restoration import denoise_tv_chambolle, denoise_bilateral


IMG_WIDTH = 352
IMG_HEIGHT = 352
IMG_CHANNELS = 3

# s-a ales din setul de date de 468 img 75% pt antrenare si 25% pentru testare

TRAIN_PATH = 'archive/segmentation/train/'
TEST_PATH = 'archive/segmentation/test/'

seed = 42
np.random.seed = seed

train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool_)

print(len(test_ids))

108


In [120]:
# Se impune augmentarea

import albumentations as A
import cv2

transform = A.Compose([
    A.RandomCrop(width=500, height=300),
    A.HorizontalFlip(p=0.7),
    A.RandomBrightnessContrast(p=0.2),
])

In [ ]:
# Make the matrix for the train images
number = np.zeros(len(train_ids))
i = 0
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    numm = id_.split('_')
    num = np.delete(numm, 1 , 0)
    number[i] = max(num)
    print(number)
    i = i + 1 
    

In [ ]:
# Get the max number of the matrix
print(number)
maxi = np.max(number)
maxi = int(maxi)
print(maxi)

In [114]:
# Make the matrix for the test images
number = np.zeros(len(test_ids))
i = 0
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    numm = id_.split('_')
    num = np.delete(numm, 1 , 0)
    number[i] = max(num)
    print(number)
    i = i + 1 
    

100%|███████████████████████████████████████████| 108/108 [00:00<00:00, 2480.87it/s]

[384.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[384. 363.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.

In [115]:
# Get the max number of the matrix
print(number)
maxi = np.max(number)
maxi = int(maxi)
print(maxi)

[384. 363. 369. 346. 348. 323. 329. 342. 380. 367. 358. 333. 350. 339.
 352. 316. 390. 377. 394. 318. 373. 379. 371. 356. 335. 337. 345. 347.
 366. 387. 360. 381. 366. 383. 364. 320. 341. 391. 399. 315. 374. 317.
 353. 330. 338. 351. 359. 336. 355. 334. 397. 370. 313. 395. 319. 344.
 361. 386. 365. 382. 367. 388. 340. 321. 375. 337. 392. 314. 400. 398.
 358. 350. 331. 335. 337. 354. 318. 373. 371. 396. 360. 368. 387. 362.
 326. 347. 343. 349. 366. 381. 353. 330. 338. 332. 351. 359. 315. 376.
 391. 317.  62. 378. 372. 319. 176. 395. 357. 334.]
400


In [103]:
#Augment the train images
from PIL import Image

#train images
index = maxi + 1
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    
#train masks
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    mask_ = imread(path + '/masks/' + id_ + '.png')
    
# Se creeaza imaginile augmentate      
    transformed = transform(image=img, mask=mask_)
    transformed_image = transformed['image']
    transformed_mask = transformed['mask'] 
    mask = Image.fromarray(transformed_mask)
    image = Image.fromarray(transformed_image)
    
# Pentru a creea folderele  
#     folder = str(index) + '_1'
#     os.makedirs(os.path.join(TRAIN_PATH + folder + '/masks'))
#     os.makedirs(os.path.join(TRAIN_PATH + folder + '/images'))

# Pentru a distribui si salva datele in fisiere       
#     folder = str(index) + '_1'   
#     mask.save( TRAIN_PATH + folder + '/masks/'+ folder + ".png")
#     image.save(TRAIN_PATH + folder + '/images/'+ folder +".png")
    
    index = index + 1
    
    

  0%|                                                       | 0/360 [00:00<?, ?it/s]/var/folders/ml/1pq8pf157qjcrdxwm5xjgzfm0000gn/T/ipykernel_3185/24374510.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
100%|█████████████████████████████████████████████| 360/360 [00:09<00:00, 36.78it/s]


In [121]:
# Augment the test images
from PIL import Image

#test images
index = maxi + 1
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    
#test masks
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    mask_ = imread(path + '/masks/' + id_ + '.png')
    
# Se creeaza imaginile augmentate   
    transformed = transform(image=img, mask=mask_)
    transformed_image = transformed['image']
    transformed_mask = transformed['mask']  
    mask = Image.fromarray(transformed_mask)
    image = Image.fromarray(transformed_image)
    
# Pentru a creea folderele  
#     folder = str(index) + '_1'
#     os.makedirs(os.path.join(TEST_PATH + folder + '/masks'))
#     os.makedirs(os.path.join(TEST_PATH + folder + '/images'))

# Pentru a distribui si salva datele in fisiere    
#     folder = str(index) + '_1'   
#     mask.save( TEST_PATH + folder + '/masks/'+ folder + ".png")
#     image.save(TEST_PATH + folder + '/images/'+ folder +".png")

    index = index + 1
    
    

  0%|                                                       | 0/108 [00:00<?, ?it/s]/var/folders/ml/1pq8pf157qjcrdxwm5xjgzfm0000gn/T/ipykernel_3185/1050119870.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
100%|█████████████████████████████████████████████| 108/108 [00:02<00:00, 39.73it/s]
